In [1]:
import pandas as pd
import numpy as np
import flow

df = flow.get_state_fips_data()
df.head()

,STATE,STATEFIPS
0,AL,01
1,AK,02
2,AZ,04
3,AR,05
4,CA,06
